In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader, TensorDataset
import os
import numpy as np
import pandas as pd
from google.colab import drive

In [2]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/chatbot-project')

Mounted at /content/drive


In [3]:
my_data = pd.read_csv('./preprocessed_qna_final.csv')
my_data.head(10)

,questions,answers
0,대성각의 대표적인 메뉴를 알 수 있나요?,"대성각의 메뉴에는 짜장면, 짬뽕이 있습니다"
1,대성각의 영업시간은 어떻게 되나요?,대성각의 영업시간은 11:00 - 15:00/21:00입니다
2,대성각의 연락처를 알 수 있나요?,대성각의 연락처는 02-356-2194입니다
3,대성각에 인접한 시설을 알 수 있나요?,"대성각에 인접한 시설에는 역촌역 3번출구, 그림나라아동미술, 메리피아노음악교습소가 ..."
4,대성각의 휴무일을 알 수 있나요?,대성각의 휴무일은 매주 일요일입니다
5,대성각의 위치를 알 수 있나요?,대성각의 위치는 서울 은평구 녹번로 7입니다
6,대성각의 주차시설이 있나요?,대성각의 주차시설은 없습니다
7,대성각의 주차시설이 있는지 알 수 있나요?,대성각의 주차시설은 없습니다
8,대성각에 인접한 시설이 있나요?,"대성각에 인접한 시설에는 역촌역 3번출구, 그림나라아동미술, 메리피아노음악교습소가 ..."
9,대성각의 주요 메뉴를 알 수 있나요?,"대성각의 메뉴에는 짜장면, 짬뽕이 있습니다"


In [4]:
#!pip install sentencepiece

In [5]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.load('spm_model_16000V_B.model')

True

In [7]:
texts = ['한양숯불갈비의 주차시설이 있나요?', '한양숯불갈비의 주차시설은 없습니다', '동흥관의 대표 메뉴가 무엇인가요?', '동흥관의 메뉴는 삼선짬뽕, 짜장면, 탕수육입니다', '까치네분식 당산역점의 영업시간을 알 수 있나요?',	'까치네분식 당산역점의 영업시간은 00:00 - 24:00입니다']
texts2 = ['한양숯불갈비의 주차시설이 있나요?', '한양숯불갈비의 주차시설은 없습니다', '동흥관의 대표 메뉴가 무엇인가요?', '동흥관의 메뉴는 삼선짬뽕, 짜장면, 탕수육입니다', '까치네분식 당산역점의 영업시간을 알 수 있나요?',	'까치네분식 당산역점의 영업시간은 00:00 - 24:00입니다']

for text in texts:
    print(f"원본 문장: {text}")
    tokens = sp.encode_as_pieces(text)
    print(f"토큰화 결과: {tokens}")
    print()

for text2 in texts2:
    print(f"원본 문장: {text2}")
    tokens2 = sp.encode_as_ids(text2)
    print(f"토큰화 결과: {tokens2}")
    print()

원본 문장: 한양숯불갈비의 주차시설이 있나요?
토큰화 결과: ['▁한', '양', '숯불갈비의', '▁주차시설이', '▁있나요', '?']

원본 문장: 한양숯불갈비의 주차시설은 없습니다
토큰화 결과: ['▁한', '양', '숯불갈비의', '▁주차시설은', '▁없습니다']

원본 문장: 동흥관의 대표 메뉴가 무엇인가요?
토큰화 결과: ['▁동', '흥', '관의', '▁대표', '▁메뉴가', '▁무엇인가요', '?']

원본 문장: 동흥관의 메뉴는 삼선짬뽕, 짜장면, 탕수육입니다
토큰화 결과: ['▁동', '흥', '관의', '▁메뉴는', '▁삼선', '짬뽕', ',', '▁짜장면', ',', '▁탕수육', '입니다']

원본 문장: 까치네분식 당산역점의 영업시간을 알 수 있나요?
토큰화 결과: ['▁까치네', '분식', '▁당산역점의', '▁영업시간을', '▁알', '▁수', '▁있나요', '?']

원본 문장: 까치네분식 당산역점의 영업시간은 00:00 - 24:00입니다
토큰화 결과: ['▁까치네', '분식', '▁당산역점의', '▁영업시간은', '▁00:00', '▁-', '▁24:00', '입니다']

원본 문장: 한양숯불갈비의 주차시설이 있나요?
토큰화 결과: [2701, 2643, 1039, 34, 1188, 73, 410, 13]

원본 문장: 한양숯불갈비의 주차시설은 없습니다
토큰화 결과: [2701, 2643, 1039, 34, 1188, 73, 410, 13]

원본 문장: 동흥관의 대표 메뉴가 무엇인가요?
토큰화 결과: [2701, 2643, 1039, 34, 1188, 73, 410, 13]

원본 문장: 동흥관의 메뉴는 삼선짬뽕, 짜장면, 탕수육입니다
토큰화 결과: [2701, 2643, 1039, 34, 1188, 73, 410, 13]

원본 문장: 까치네분식 당산역점의 영업시간을 알 수 있나요?
토큰화 결과: [2701, 2643, 1039, 34, 1188, 73, 410, 13]

원본 문장: 까치네분식 당산역점의 영업시간은 

In [9]:
questions = my_data['questions']
answers = my_data['answers']

# 토큰화
question_tokens = [sp.encode_as_ids(q, add_bos=True, add_eos=True) for q in questions]
answer_tokens = [sp.encode_as_ids(a, add_bos=True, add_eos=True) for a in answers]

# 토큰화된 리스트를 PyTorch 텐서로 변환
questions_tensors = [torch.tensor(t, dtype=torch.long) for t in question_tokens]
answers_tensors = [torch.tensor(t, dtype=torch.long) for t in answer_tokens]

# 패딩을 위한 최대 길이 결정
questions_max_length = max(tensor.size(0) for tensor in questions_tensors)
answers_max_length = max(tensor.size(0) for tensor in answers_tensors)
max_length = max(questions_max_length, answers_max_length)

In [10]:
# 패딩 함수
def pad_or_truncate(tensors, max_length, pad_token_id):
    padded_tensors = []
    for tensor in tensors:
        if tensor.size(0) < max_length:
            padded_tensor = torch.cat([tensor, torch.tensor([pad_token_id] * (max_length - tensor.size(0)), dtype=torch.long)])
        else:
            padded_tensor = tensor[:max_length]
        padded_tensors.append(padded_tensor)
    return pad_sequence(padded_tensors, batch_first=True)

In [13]:
pad_token_id = sp.pad_id()
unk_token_id = sp.unk_id()
bos_token_id = sp.bos_id()
eos_token_id = sp.eos_id()
print("Pad Token ID:", pad_token_id)
print("Unk Token ID:", unk_token_id)
print("BOS Token ID:", bos_token_id)
print("EOS Token ID:", eos_token_id)

Pad Token ID: 0
Pad Token ID: 1
BOS Token ID: 2
EOS Token ID: 3


In [14]:
# 패딩 적용
questions_padded = pad_or_truncate(questions_tensors, max_length, pad_token_id=pad_token_id)
answers_padded = pad_or_truncate(answers_tensors, max_length, pad_token_id=pad_token_id)

# 패딩된 텐서의 모양
print("Padded Questions Shape:", questions_padded.shape)
print("Padded Answers Shape:", answers_padded.shape)

Padded Questions Shape: torch.Size([26691, 50])
Padded Answers Shape: torch.Size([26691, 50])


In [15]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=max_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return x

In [16]:
class TransformerModel(nn.Module):
    def __init__(self, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, vocab_size, max_len=max_length):
        super(TransformerModel, self).__init__()
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead,
                                          num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers,
                                          dim_feedforward=dim_feedforward,
                                          batch_first=True)
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len)
        self.fc_out = nn.Linear(d_model, vocab_size)

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None):
        src = self.embedding(src)
        tgt = self.embedding(tgt)
        src = self.pos_encoder(src)
        tgt = self.pos_encoder(tgt)

        tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)

        output = self.transformer(src, tgt,
                                  tgt_mask=tgt_mask,
                                  src_key_padding_mask=src_key_padding_mask,
                                  tgt_key_padding_mask=tgt_key_padding_mask)
        return self.fc_out(output)

In [17]:
class EarlyStopping:
    def __init__(self, patience=4, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.best_loss = float('inf')
        self.counter = 0
        self.stopped_early = False

    def __call__(self, train_loss, model):
        if train_loss < self.best_loss:
            self.best_loss = train_loss
            self.counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            self.counter += 1
            if self.verbose:
                print(f'Training loss did not improve. Counter: {self.counter}/{self.patience}')
            if self.counter >= self.patience:
                self.stopped_early = True
                if self.verbose:
                    print('Early stopping triggered.')

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [19]:
vocab_size = sp.get_piece_size()
print("Vocab Size:", vocab_size)
d_model = 512
nhead = 8
num_encoder_layers = 6
num_decoder_layers = 6
dim_feedforward = 2048

Vocab Size: 16000


In [20]:
# 데이터셋 생성
dataset = TensorDataset(questions_padded, answers_padded)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=2, pin_memory=True)

# 모델, 옵티마이저 생성
model = TransformerModel(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers,
                         num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward,
                         vocab_size=vocab_size, max_len=max_length)

model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=pad_token_id)

early_stopping = EarlyStopping(patience=4, verbose=True)

scaler = GradScaler()

# 학습
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for batch in dataloader:
        src_batch, tgt_batch = batch

        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)

        tgt_input = tgt_batch[:, :-1]
        tgt_output = tgt_batch[:, 1:]

        src_key_padding_mask = (src_batch == pad_token_id).to(device)
        tgt_key_padding_mask = (tgt_input == pad_token_id).to(device)

        optimizer.zero_grad()

        with autocast():
            output = model(src_batch, tgt_input,
                       src_key_padding_mask=src_key_padding_mask,
                       tgt_key_padding_mask=tgt_key_padding_mask)
            loss = criterion(output.contiguous().view(-1, vocab_size), tgt_output.contiguous().view(-1))

        scaler.scale(loss).backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(dataloader)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.6f}")

    early_stopping(avg_loss, model)
    if early_stopping.stopped_early:
        break

print("Training complete!")

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch 1/50, Train Loss: 6.602952
Epoch 2/50, Train Loss: 4.758540
Epoch 3/50, Train Loss: 3.934490
Epoch 4/50, Train Loss: 3.541922
Epoch 5/50, Train Loss: 3.277982
Epoch 6/50, Train Loss: 3.045592
Epoch 7/50, Train Loss: 2.822670
Epoch 8/50, Train Loss: 2.630520
Epoch 9/50, Train Loss: 2.441388
Epoch 10/50, Train Loss: 2.220019
Epoch 11/50, Train Loss: 1.944709
Epoch 12/50, Train Loss: 1.634103
Epoch 13/50, Train Loss: 1.320207
Epoch 14/50, Train Loss: 1.056794
Epoch 15/50, Train Loss: 0.850318
Epoch 16/50, Train Loss: 0.688536
Epoch 17/50, Train Loss: 0.561807
Epoch 18/50, Train Loss: 0.466625
Epoch 19/50, Train Loss: 0.394657
Epoch 20/50, Train Loss: 0.306186
Epoch 21/50, Train Loss: 0.260530
Epoch 22/50, Train Loss: 0.237269
Epoch 23/50, Train Loss: 0.217194
Epoch 24/50, Train Loss: 0.202062
Epoch 25/50, Train Loss: 0.191080
Epoch 26/50, Train Loss: 0.181071
Epoch 27/50, Train Loss: 0.173063
Epoch 28/50, Train Loss: 0.166791
Epoch 29/50, Train Loss: 0.161083
Epoch 30/50, Train Loss

In [24]:
def generate_text(model, src, sp, max_length=max_length, device='cuda'):
    src = src.to(device)
    src_key_padding_mask = (src == pad_token_id).to(device)

    with torch.no_grad():
        tgt_input = torch.tensor([[bos_token_id]], device=device, dtype=torch.long)
        for _ in range(max_length):
            tgt_key_padding_mask = (tgt_input == pad_token_id).to(device)

            output = model(src, tgt_input, src_key_padding_mask=src_key_padding_mask,
                           tgt_key_padding_mask=tgt_key_padding_mask)
            output_probs = F.softmax(output[:, -1, :], dim=-1)
            next_token = torch.argmax(output_probs, dim=-1)

            tgt_input = torch.cat((tgt_input, next_token.unsqueeze(0)), dim=1)

            if next_token.item() == eos_token_id:
                break

    return tgt_input[:, 1:].squeeze(0)

In [27]:
def decode_output(output_tokens, sp):
    tokens = [sp.id_to_piece(int(token)) for token in output_tokens]
    return sp.decode([token for token in tokens if token not in ['<eos>', '<bos>', '<unk>']])

In [29]:
# 평가
model.eval()

try:
    with torch.no_grad():
        # 첫 번째 배치만 평가
        batch = next(iter(dataloader))
        src_batch, tgt_batch = batch
        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)

        for i, (src_single, tgt_single) in enumerate(zip(src_batch, tgt_batch)):
            generated = generate_text(model, src_single.unsqueeze(0), sp, max_length=max_length, device=device)
            input_text = sp.decode([sp.id_to_piece(int(token)) for token in src_single.cpu().numpy() if sp.id_to_piece(int(token)) not in ['<eos>', '<bos>', '<unk>']])
            predicted_text = decode_output(generated.cpu().numpy(), sp)
            target_text = sp.decode([sp.id_to_piece(int(token)) for token in tgt_single.cpu().numpy() if sp.id_to_piece(int(token)) not in ['<eos>', '<bos>', '<unk>']])

            print(f"\nSample {i+1}:")
            print(f"Input: {input_text}")
            print(f"Predicted: {predicted_text}")
            print(f"Target: {target_text}")

            if i == 9:
                break

except Exception as e:
    print(f"An error occurred during evaluation: {str(e)}")

print("Evaluation complete!")


Sample 1:
Input: 도레도레 롯데몰김포공항점의 위치는 어떻게 되나요?
Predicted: 도레도레 롯데몰김포공항점의 위치는 서울 강서구 하늘길 77입니다
Target: 도레도레 롯데몰김포공항점의 위치는 서울 강서구 하늘길 77입니다

Sample 2:
Input: 영등포구 맛집 추천좀
Predicted: 영등포구의 맛집으로는 춘뽕이 알려져 있습니다
Target: 영등포구의 맛집으로는 숯닭가 있습니다

Sample 3:
Input: 삼성원의 연락처를 알 수 있나요?
Predicted: 삼성원의 연락처는 02-2634-1140입니다
Target: 삼성원의 연락처는 02-2634-1140입니다

Sample 4:
Input: 카페 놀이터의 영업시간은 어떻게 되나요?
Predicted: 명동찌개 불광점의 연락처는 02-387-9269입니다
Target: 카페 놀이터의 영업시간은 10:30 - 22:00입니다

Sample 5:
Input: 송파구 맛집 추천좀
Predicted: 송파구의 맛집으로는 싸다김밥 석촌역점이 있습니다
Target: 송파구의 맛집으로는 바다풍경를 추천드립니다

Sample 6:
Input: 영등포구 맛집 추천해줄래?
Predicted: 영등포구의 맛집으로는 이화의밤에 가보세요
Target: 영등포구의 맛집으로는 대관원가 알려져 있습니다

Sample 7:
Input: 보울레시피 신용산점의 휴무일은 어떻게 되나요?
Predicted: 보울레시피 신용산점의 인접한 시설에는 신용산역 3번 출구, 감성커피, 스타벅스, 문지스뷰티, 하카타라멘이 있습니다
Target: 보울레시피 신용산점의 휴무일은 없습니다

Sample 8:
Input: 슈퍼커피 독산점의 주차시설이 있나요?
Predicted: 슈퍼커피 독산점의 인접한 시설에는 연신내역 5번 출구가 있습니다
Target: 슈퍼커피 독산점의 주차시설은 있습니다

Sample 9:
Input: 공원분식의 영업시간은 어떻게 되나요?
Predicted: 공원분식의 연락처는 알수없습니다
Target:

In [30]:
dataloader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# 학습
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for batch in dataloader:
        src_batch, tgt_batch = batch

        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)

        tgt_input = tgt_batch[:, :-1]
        tgt_output = tgt_batch[:, 1:]

        src_key_padding_mask = (src_batch == pad_token_id).to(device)
        tgt_key_padding_mask = (tgt_input == pad_token_id).to(device)

        optimizer.zero_grad()

        with autocast():
            output = model(src_batch, tgt_input,
                       src_key_padding_mask=src_key_padding_mask,
                       tgt_key_padding_mask=tgt_key_padding_mask)
            loss = criterion(output.contiguous().view(-1, vocab_size), tgt_output.contiguous().view(-1))

        scaler.scale(loss).backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(dataloader)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.6f}")

    early_stopping(avg_loss, model)
    if early_stopping.stopped_early:
        break

print("Training complete!")

Epoch 1/20, Train Loss: 0.108622
Epoch 2/20, Train Loss: 0.106713
Epoch 3/20, Train Loss: 0.106798
Training loss did not improve. Counter: 1/4
Epoch 4/20, Train Loss: 0.105609
Epoch 5/20, Train Loss: 0.099986
Epoch 6/20, Train Loss: 0.099581
Epoch 7/20, Train Loss: 0.099110
Epoch 8/20, Train Loss: 0.098771
Epoch 9/20, Train Loss: 0.099267
Training loss did not improve. Counter: 1/4
Epoch 10/20, Train Loss: 0.098824
Training loss did not improve. Counter: 2/4
Epoch 11/20, Train Loss: 0.098675
Epoch 12/20, Train Loss: 0.098725
Training loss did not improve. Counter: 1/4
Epoch 13/20, Train Loss: 0.098446
Epoch 14/20, Train Loss: 0.096096
Epoch 15/20, Train Loss: 0.094695
Epoch 16/20, Train Loss: 0.094671
Epoch 17/20, Train Loss: 0.094406
Epoch 18/20, Train Loss: 0.094468
Training loss did not improve. Counter: 1/4
Epoch 19/20, Train Loss: 0.094096
Epoch 20/20, Train Loss: 0.094260
Training loss did not improve. Counter: 1/4
Training complete!


In [31]:
# 평가
model.eval()

try:
    with torch.no_grad():
        # 첫 번째 배치만 평가
        batch = next(iter(dataloader))
        src_batch, tgt_batch = batch
        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)

        for i, (src_single, tgt_single) in enumerate(zip(src_batch, tgt_batch)):
            generated = generate_text(model, src_single.unsqueeze(0), sp, max_length=max_length, device=device)
            input_text = sp.decode([sp.id_to_piece(int(token)) for token in src_single.cpu().numpy() if sp.id_to_piece(int(token)) not in ['<eos>', '<bos>', '<unk>']])
            predicted_text = decode_output(generated.cpu().numpy(), sp)
            target_text = sp.decode([sp.id_to_piece(int(token)) for token in tgt_single.cpu().numpy() if sp.id_to_piece(int(token)) not in ['<eos>', '<bos>', '<unk>']])

            print(f"\nSample {i+1}:")
            print(f"Input: {input_text}")
            print(f"Predicted: {predicted_text}")
            print(f"Target: {target_text}")

            if i == 9:
                break

except Exception as e:
    print(f"An error occurred during evaluation: {str(e)}")

print("Evaluation complete!")


Sample 1:
Input: 꼴목포차 응암점의 영업시간은 어떻게 되나요?
Predicted: 꼴목포차 응암점의 인접한 시설에는 17:00 - 04:00/05:00입니다
Target: 꼴목포차 응암점의 영업시간은 17:00 - 04:00/05:00입니다

Sample 2:
Input: 인어나라의 주요 메뉴가 어떻게 되나요?
Predicted: 인어나라의 맛집으로는 카페 엔드롱가 알려져 있습니다
Target: 인어나라의 메뉴에는 광어, 생연어가 있습니다

Sample 3:
Input: 강서구 맛집 추천해줘
Predicted: 강서구의 맛집으로는 요거프레소 화곡푸르지오점에 가보세요
Target: 강서구의 맛집으로는 김순옥들내음를 추천드립니다

Sample 4:
Input: 도화루중화요리에 인접한 시설을 알 수 있나요?
Predicted: 금천구의 맛집으로는 쉬즈베이글 금천구점이 있습니다
Target: 도화루중화요리에 인접한 시설에는 송정역 3번 출구, 까치식당, 세븐일레븐가 있습니다

Sample 5:
Input: 춘자싸롱에 인접한 시설이 있나요?
Predicted: 구로구의 맛집으로는 멕시칸치킨 신도림동아점를 추천드립니다
Target: 춘자싸롱에 인접한 시설에는 구로디지털단지역 2번 출구, 여기가전집, 세광양대창이 있습니다

Sample 6:
Input: 북촌손만두 대림점의 주차시설이 있나요?
Predicted: 북촌손만두 대림점의 인접한 시설에는 시흥대로 671 1번 출구, 시흥대로 671 있습니다
Target: 북촌손만두 대림점의 주차시설은 없습니다

Sample 7:
Input: 한입소반의 영업시간을 알 수 있나요?
Predicted: 한입소반의 위치는 서울 용산구 청파로45길 3입니다
Target: 한입소반의 영업시간은 07:00 - 19:00입니다

Sample 8:
Input: 관악구 맛집을 알고싶어
Predicted: 관악구의 맛집으로는 스시락를 추천드립니다
Target: 관악구의 맛집으로는 로우포지션가 있습니다

Sample 9:
Input: 봉

In [32]:
def chatbot(model, sp, device):
    while True:
        question = input("질문을 입력하세요 (종료하려면 '나가기' 입력): ")
        if question.strip() == '나가기':
            print("프로그램을 종료합니다.")
            break

        # 질문을 토크나이즈
        input_ids = torch.tensor(sp.EncodeAsIds(question)).unsqueeze(0).to(device)

        # 응답 생성
        response_ids = generate_text(model, input_ids, sp, device=device)

        # 응답 디코딩
        response = sp.decode([sp.id_to_piece(int(token)) for token in response_ids.cpu().numpy() if sp.id_to_piece(int(token)) not in ['<eos>', '<bos>', '<unk>']])

        print(f"답변: {response}\n")

# 모델과 SentencePiece 모델, 그리고 디바이스 설정을 전달
chatbot(model, sp, device)

질문을 입력하세요 (종료하려면 '나가기' 입력): 강서구 맛집 추천해줘


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


답변: 강서구의 맛집으로는 고요베이크샵이 있습니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 고요베이크샵은 메뉴가 뭐가 있어?
답변: 보리밥 손수제비의 메뉴에는 보리밥, 손수제비가 있습니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 고요베이크샵은 주차시설이 있어?
답변: 색조화 서울 강서구의 주차시설은 있습니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 고요베이크샵 위치
답변: 금천구의 맛집으로는 쉬즈베이글 금천구점이 있습니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 나가기
프로그램을 종료합니다.


In [ ]:
model = TransformerModel(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers,
                          num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward,
                          vocab_size=vocab_size, max_len=max_length)

model.to(device)

model.load_state_dict(torch.load('best_transformer_model.pth'))
model.eval()

chatbot(model, tokenizer, device)

질문을 입력하세요 (종료하려면 '나가기' 입력): 성북구 맛집 추천
답변: 성북구의 맛집으로는 메이크미해피가 있습니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 메이크미해피 메뉴
답변: 메이크미해피의 메뉴에는 맥미라떼, 맥미해피가 있습니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 메이크미해피 주차시설
답변: 메이크미해피의 주차시설은 있습니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 메이크미해피 인접 시설
답변: 메이크미해피에 인접한 시설에는 안암역 3번 출구, 카페 브레송, 청원가 있습니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 성북구 다른 맛집 추천해줘
답변: 성북구의 맛집으로는 로스터리까페라운드테이블가 알려져 있습니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 로스터리까페라운드테이블 위치가 어디야
답변: 로스터리까페라운드테이블의 위치는 서울 성북구 보문로30길 40-2입니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 로스터리까페라운드테이블은 뭐가 유명해?
답변: 로스터리까페라운드테이블의 연락처는 02-923-7573입니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 로스터리까페라운드테이블 휴무일 언제야?
답변: 로스터리까페라운드테이블의 휴무일은 알수없습니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 로스터리까페라운드테이블 영업시간 알려줄 수 있어?
답변: 로스터리까페라운드테이블의 영업시간은 10:00~22:00입니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 성북구 또다른 맛집 알려줘
답변: 성북구의 맛집으로는 밀키웨이가 있습니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 밀키웨이 메뉴 추천좀
답변: 타웨이커스의 메뉴에는 타코웨이코 타코 타코 타코 타코 타코 타코 타코 타코 타코 데 세일식샌드위식 상요일 타코 타코 타코 데에멘의 메뉴에는 타코 타코 타코 타코 타코 타코 타코 타코 타코 타코

질문을 입력하세요 (종료하려면 '나가기' 입력): 성북구 맛집을 알고싶은